In [133]:
from typing import Dict, List
import pandas as pd
import datetime as dt
import os
import re
import json
from dotenv import load_dotenv
import pandas as pd

from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Point, Line
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

In [134]:
speckle_server = "app.speckle.systems"
speckle_token = os.environ["TOKEN"]
client = SpeckleClient(host=speckle_server)
client.authenticate_with_token(speckle_token)
stream_id = "013aaf06e7"
transport = ServerTransport(client=client, stream_id=stream_id)

In [135]:

object_id = "eb80959eb9eebaafb7a4f1db38e841d1"
obj = client.object.get(stream_id, object_id)



In [136]:
from specklepy.serialization.base_object_serializer import BaseObjectSerializer
serializer = BaseObjectSerializer()

In [137]:
hash_id, obj_data = serializer.traverse_base(obj)
obj_data["userStrings"] 

{'id': 'f3758c29e2daf3477efd27e414e5fb8d',
 'speckle_type': 'Base',
 'totalChildrenCount': 0,
 'Bottom z (mPD)': -50,
 'Domain Margin (m)': 20,
 'Max Easting (m)': 838845,
 'Max Northing (m)': 821245,
 'Min Easting (m)': 837755,
 'Min Northing (m)': 819780,
 'Top z (mPD)': 0,
 'Voxel Size (m)': 5,
 'applicationId': None,
 'units': None}

In [138]:
x_min = obj_data["userStrings"]['Min Easting (m)']
x_max = obj_data["userStrings"]['Max Easting (m)']
y_min = obj_data["userStrings"]['Min Northing (m)']
y_max = obj_data["userStrings"]['Max Northing (m)']
z_min = obj_data["userStrings"]['Bottom z (mPD)']
z_max = obj_data["userStrings"]['Top z (mPD)']
voxel_size = obj_data["userStrings"]["Voxel Size (m)"]

x_points = int((x_max - x_min) / voxel_size)
y_points = int((y_max - y_min) / voxel_size)
z_points = int((z_max - z_min) / voxel_size)

x_points, y_points, z_points


(218, 293, 10)

In [139]:
object_id = "1a98b24773c42b01be65b71f63ee14bc"
start_end_points = operations.receive(object_id, transport)

In [140]:
_, start_end_points_data = serializer.traverse_base(start_end_points)
with open("start_end_points.json", "w") as f:
    json.dump(start_end_points_data, f)

In [141]:
start_point = start_end_points_data["elements"][1]["elements"][0]
end_points = start_end_points_data["elements"][0]

start = {
    "x" : start_point["x"],
    "y" : start_point["y"],
    "z" : start_point["z"]
}
end_points = [
    {
        "x" : end_point["x"],
        "y" : end_point["y"],
        "z" : end_point["z"]
    }
    for end_point in end_points["elements"]
]

In [142]:
end_points

[{'x': 838545.0, 'y': 821225.0, 'z': 0.0},
 {'x': 837775.0, 'y': 820755.0, 'z': 0.0}]

In [143]:
start_index = {
    "x" : int((start["x"] - x_min) / voxel_size),
    "y" : int((start["y"] - y_min) / voxel_size),
    "z" : int((start["z"] - z_min) / voxel_size)
}

In [144]:
start_index

{'x': 214, 'y': 4, 'z': 10}

In [146]:
end_indices = [
    {
        "x" : int((end_point["x"] - x_min) / voxel_size),
        "y" : int((end_point["y"] - y_min) / voxel_size),
        "z" : int((end_point["z"] - z_min) / voxel_size)
    }
    for end_point in end_points
]



({'x': 214, 'y': 4, 'z': 10},
 [{'x': 158, 'y': 289, 'z': 10}, {'x': 4, 'y': 195, 'z': 10}])

In [ ]:
# In summary the data is as follows:



